# Выбор локации для скважины

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. 

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Подготовка данных первого региона

In [2]:
# чтение и просмотр файла
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
# изменение регистра данных в столбце id
df_0['id'] = df_0['id'].str.lower()
df_0.head()

,id,f0,f1,f2,product
0,txeyh,0.705745,-0.497823,1.221170,105.280062
1,2acmu,1.334711,-0.340164,4.365080,73.037750
2,409wp,1.022732,0.151990,1.419926,85.265647
3,ijlyr,-0.032172,0.139033,2.978566,168.620776
4,xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
# просмотр информации 
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
# поиск явных дубликатов
df_0.duplicated().sum()

0

In [6]:
# поиск неявных дубликатов
df_0['id'].value_counts()

qw6lh    2
yfcgc    2
xudjb    2
ocw1d    2
tdehs    2
        ..
ocz8x    1
jfgjd    1
lrbr0    1
15tlk    1
dwtjw    1
Name: id, Length: 99887, dtype: int64

#### Вывод

- ознакомлена с данными первой таблицы
- данные в столбце `id` приведены к одному нижнему регистру
- пропусков, явных дубликатов не обнаружено
- обнаружены неявные дубликаты, оставлены без изменений, так как неизвестна природа их возникновения
- типы данных в столбцах соответствуют действительности

### Подготовка данных второго региона

In [7]:
# чтение и просмотр файла
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [8]:
# изменение регистра данных в столбце id
df_1['id'] = df_1['id'].str.lower()
df_1.head()

,id,f0,f1,f2,product
0,kbedx,-15.001348,-8.276000,-0.005876,3.179103
1,62mp7,14.272088,-3.475083,0.999183,26.953261
2,vye1p,6.263187,-5.948386,5.001160,134.766305
3,kcrkz,-13.081196,-11.506057,4.999415,137.945408
4,ahl4o,12.702195,-8.147433,5.004363,134.766305


In [9]:
# просмотр информации 
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
# поиск явных дубликатов
df_1.duplicated().sum()

0

In [11]:
# поиск неявных дубликатов
df_1['id'].value_counts()

zyegi    2
mmvjw    2
afdl4    2
ocw8l    2
2lern    2
        ..
ldghi    1
amfdg    1
d5b6l    1
75eed    1
rhz5l    1
Name: id, Length: 99887, dtype: int64

#### Вывод

- ознакомлена с данными первой таблицы
- данные в столбце `id` приведены к одному нижнему регистру
- пропусков, явных дубликатов не обнаружено
- обнаружены неявные дубликаты, оставлены без изменений, так как неизвестна природа их возникновения
- типы данных в столбцах соответствуют действительности

### Подготовка данных третьего региона

In [12]:
# чтение и просмотр файла
df_2 = pd.read_csv('/datasets/geo_data_2.csv')
df_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [13]:
# изменение регистра данных в столбце id
df_2['id'] = df_2['id'].str.lower()
df_2.head()

,id,f0,f1,f2,product
0,fwxo0,-1.146987,0.963328,-0.828965,27.758673
1,wjtft,0.262778,0.269839,-2.530187,56.069697
2,ovluw,0.194587,0.289035,-5.586433,62.871910
3,q6ca6,2.236060,-0.553760,0.930038,114.572842
4,wpmux,-0.515993,1.716266,5.899011,149.600746


In [14]:
# просмотр информации 
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [15]:
# поиск явных дубликатов
df_2.duplicated().sum()

0

In [16]:
# поиск неявных дубликатов
df_2['id'].value_counts()

nxgix    2
oiwjq    2
q6rkz    2
0jrjy    2
wlbmd    2
        ..
u4na0    1
icweu    1
qn0fk    1
y2jtr    1
pv0gm    1
Name: id, Length: 99901, dtype: int64

#### Вывод

- ознакомлена с данными первой таблицы
- данные в столбце `id` приведены к одному нижнему регистру
- пропусков, явных дубликатов не обнаружено
- обнаружены неявные дубликаты, оставлены без изменений, так как неизвестна природа их возникновения
- типы данных в столбцах соответствуют действительности

## Обучение и проверка модели

In [17]:
# значение random_state выносим в константу
RANDOM_STATE = 12345

### Обучение и проверка модели первого региона

#### Разбиение на выборки

In [18]:
# выявление целевого признака и остальных признаков
# удаление столбца id для исключения выявления лишней зависимости
target_0 = df_0['product']
features_0 = df_0.drop(['product','id'], axis=1)

In [19]:
# деление исходного набора данных на две выборки: обучающую и валидационную
# в соотношении 75:25
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0,
                                                                                      target_0,
                                                                                      test_size = 0.25,
                                                                                      random_state=RANDOM_STATE)

In [20]:
# проверка корректности разбиения
print(features_train_0.shape, 
      features_valid_0.shape)

(75000, 3) (25000, 3)


#### Масштабирование признаков

In [21]:
# масштабирование численных признаков
pd.options.mode.chained_assignment = None
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(features_train_0[numeric])
features_train_0[numeric] = scaler.transform(features_train_0[numeric])
features_valid_0[numeric] = scaler.transform(features_valid_0[numeric])

#### Обучение и предсказание модели

In [22]:
model = LinearRegression()
model.fit(features_train_0, target_train_0)
predictions_valid_0 = model.predict(features_valid_0)
result_0 = mean_squared_error(target_valid_0, predictions_valid_0)

In [23]:
print('Средний запас предсказанного сырья:', predictions_valid_0.mean())
print('RMSE модели линейной регрессии на валидационной выборке', result_0**0.5)

Средний запас предсказанного сырья: 92.59256778438035
RMSE модели линейной регрессии на валидационной выборке 37.5794217150813


### Обучение и проверка модели второго региона

#### Разбиение на выборки

In [24]:
# выявление целевого признака и остальных признаков
# удаление столбца id для исключения выявления лишней зависимости
target_1 = df_1['product']
features_1 = df_1.drop(['product','id'], axis=1)

In [25]:
# деление исходного набора данных на две выборки: обучающую и валидационную
# в соотношении 75:25
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1,
                                                                                      target_1,
                                                                                      test_size = 0.25,
                                                                                      random_state=RANDOM_STATE)

In [26]:
# проверка корректности разбиения
print(features_train_1.shape, 
      features_valid_1.shape)

(75000, 3) (25000, 3)


#### Масштабирование признаков

In [27]:
# масштабирование численных признаков
pd.options.mode.chained_assignment = None
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(features_train_1[numeric])
features_train_1[numeric] = scaler.transform(features_train_1[numeric])
features_valid_1[numeric] = scaler.transform(features_valid_1[numeric])

#### Обучение и предсказание модели

In [28]:
model = LinearRegression()
model.fit(features_train_1, target_train_1)
predictions_valid_1 = model.predict(features_valid_1)
result_1 = mean_squared_error(target_valid_1, predictions_valid_1)

In [29]:
print('Средний запас предсказанного сырья:', predictions_valid_1.mean())
print('RMSE модели линейной регрессии на валидационной выборке', result_1**0.5)

Средний запас предсказанного сырья: 68.728546895446
RMSE модели линейной регрессии на валидационной выборке 0.893099286775617


### Обучение и проверка модели третьего региона

#### Разбиение на выборки

In [30]:
# выявление целевого признака и остальных признаков
# удаление столбца id для исключения выявления лишней зависимости
target_2 = df_2['product']
features_2 = df_2.drop(['product','id'], axis=1)

In [31]:
# деление исходного набора данных на две выборки: обучающую и валидационную
# в соотношении 75:25
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2,
                                                                                      target_2,
                                                                                      test_size = 0.25,
                                                                                      random_state=RANDOM_STATE)

In [32]:
# проверка корректности разбиения
print(features_train_2.shape, 
      features_valid_2.shape)

(75000, 3) (25000, 3)


#### Масштабирование признаков

In [33]:
# масштабирование численных признаков
pd.options.mode.chained_assignment = None
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(features_train_2[numeric])
features_train_2[numeric] = scaler.transform(features_train_2[numeric])
features_valid_2[numeric] = scaler.transform(features_valid_2[numeric])

#### Обучение и предсказание модели

In [34]:
model = LinearRegression()
model.fit(features_train_2, target_train_2)
predictions_valid_2 = model.predict(features_valid_2)
result_2 = mean_squared_error(target_valid_2, predictions_valid_2)

In [35]:
print('Средний запас предсказанного сырья:', predictions_valid_2.mean())
print('RMSE модели линейной регрессии на валидационной выборке', result_2**0.5)

Средний запас предсказанного сырья: 94.96504596800489
RMSE модели линейной регрессии на валидационной выборке 40.02970873393434


### Вывод

In [36]:
predict_tab = [predictions_valid_0.mean(), predictions_valid_1.mean(), predictions_valid_2.mean()]
rmse_tab = [result_0**0.5, result_1**0.5, result_2**0.5]
print(predict_tab)
print(rmse_tab)

[92.59256778438035, 68.728546895446, 94.96504596800489]
[37.5794217150813, 0.893099286775617, 40.02970873393434]


Средний запас предсказанного сырья больше в третьем регионе. Лучшее качество показала модель второго региона.

## Подготовка к расчёту прибыли

In [37]:
# Бюджет на разработку скважин в регионе — 10 млрд рублей
# Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей
# При разведке региона выбирают 200 точек
BUDGET = 10_000_000_000
ONE_PRODUCT = 450_000
POINTS = 200

In [38]:
print('Достаточный объём сырья для безубыточной разработки новой скважины:', BUDGET/(ONE_PRODUCT*POINTS))

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111


**Вывод:**

Предсказанный объем сырья в первом и третьем регионе приближен к полученному значению достаточного объёма.

In [51]:
def profit(target, predictions):
    best_pred = pd.Series(predictions).sort_values(ascending=False)[:POINTS]
    best_target = target[best_pred.index]
    return best_target.sum() * ONE_PRODUCT - BUDGET

## Расчёт прибыли и рисков 

In [52]:
state = np.random.RandomState(12345)

In [53]:
def region_profit(target, predictions):    
    values = []
    for i in range(1000):
        subsample = target.sample(500,
                                  replace=True,
                                  random_state=state)
        best_subsample = pd.Series(predictions, index=target.index)[subsample.index]
        prof = profit(subsample, best_subsample)
        values.append(prof)
    values = pd.Series(values)
    mean = values.mean() 
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    print('Средняя прибыль в регионе:', mean)
    print('0.025-квантиль:', lower)
    print('0.0975-квантиль:', upper)
    print('Доля отрицательной прибыли:',(values < 0).mean())

In [54]:
region_profit(target_valid_0, predictions_valid_0)
region_profit(target_valid_1, predictions_valid_1)
region_profit(target_valid_2, predictions_valid_2)

Средняя прибыль в регионе: 600735244.2611653
0.025-квантиль: 12948331.135115242
0.0975-квантиль: 1231163605.7914982
Доля отрицательной прибыли: 0.02
Средняя прибыль в регионе: 663958995.2601906
0.025-квантиль: 206476361.25177094
0.0975-квантиль: 1191197684.7488434
Доля отрицательной прибыли: 0.001
Средняя прибыль в регионе: 597381047.9005232
0.025-квантиль: 1734929.5311760982
0.0975-квантиль: 1246217960.1652355
Доля отрицательной прибыли: 0.025


### Вывод

Вероятность убытков меньше 2.5% обнаружена в первом и во втором регионе. Вероятность убытков в первом регионе равна 2%, во втором - 0.1%. Наибольшая средняя прибыль во втором регионе. Можно сделать вывод, что выбрать для разработки скважен перспективнее второй регион.